# Using Knowledge Graph with Neo4jGraphStore


In [1]:
from llama_index.readers import file
print(dir(file))

['CSVReader', 'DocxReader', 'EpubReader', 'FlatReader', 'HTMLTagReader', 'HWPReader', 'IPYNBReader', 'ImageCaptionReader', 'ImageReader', 'ImageTabularChartReader', 'ImageVisionLLMReader', 'MarkdownReader', 'MboxReader', 'PDFReader', 'PagedCSVReader', 'PandasCSVReader', 'PandasExcelReader', 'PptxReader', 'PyMuPDFReader', 'RTFReader', 'UnstructuredReader', 'VideoAudioReader', 'XMLReader', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'docs', 'epub', 'flat', 'html', 'image', 'image_caption', 'image_deplot', 'image_vision_llm', 'ipynb', 'markdown', 'mbox', 'paged_csv', 'pymu_pdf', 'rtf', 'slides', 'tabular', 'unstructured', 'video_audio', 'xml']


In [3]:
from llama_index.core import KnowledgeGraphIndex, SimpleDirectoryReader
from llama_index.core import StorageContext
from llama_index.graph_stores.neo4j import Neo4jGraphStore
import logging
import sys
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from IPython.display import Markdown, display
import os
from llama_index.readers.s3 import S3Reader
import time
from neo4j import GraphDatabase

## Configurando LLM

In [8]:
os.environ["OPENAI_API_KEY"] = 'chave_api'

logging.basicConfig(stream=sys.stdout, level=logging.INFO)

# define LLM
llm = OpenAI(temperature=0, api_key = 'chave_api',model="gpt-4o-mini")
Settings.llm = llm
Settings.chunk_size = 512

## Carregando documentos

## Processando os arquivos

## Indexando documentos no Neo4j

In [9]:
# Credenciais de Acesso ao banco de dados de grafos Neo4j
username = "neo4j"
password = 'senha_neo4j'
url = 'uri_neo4j'
database = "neo4j"
driver = GraphDatabase.driver(url, auth=(username, password))

In [10]:
graph_store = Neo4jGraphStore(
    username=username,
    password=password,
    url=url,
    database=database,
)

In [11]:
from llama_index.core.schema import Document

# Caminho onde estão os arquivos .txt
folder_path = "/teamspace/studios/this_studio/SIRI_BIA/Dados"

# Carregar arquivos como documentos
documents = []
for filename in os.listdir(folder_path):
    if filename.endswith(".txt"):
        with open(os.path.join(folder_path, filename), "r", encoding="utf-8") as f:
            content = f.read()
            documents.append(Document(text=content))

In [12]:
inicio = time.time()

storage_context = StorageContext.from_defaults(graph_store=graph_store)

index = KnowledgeGraphIndex.from_documents(
    documents,
    storage_context=storage_context,
    max_triplets_per_chunk=4,
    include_embeddings=True
)

fim = time.time()

tempo_decorrido = fim - inicio

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST

In [14]:
print(f"Tempo para indexação de {len(documents)} documentos: {tempo_decorrido/60:.2f} minutos")

Tempo para indexação de 5 documentos: 7.47 minutos


## Queries

### RAG

Iniciaremos com testes de recuperação + geração de resposta (LLM)

In [23]:
query_engine = index.as_query_engine(
    include_text=False,
    response_mode="tree_summarize", # include_text =True send the text from where the retrieved tripets were extracted.
    embedding_mode="hybrid",
    similarity_top_k=5,
)

In [24]:
inicio = time.time()
response_1 = query_engine.query("Quantas matérias tem o curso?")
fim = time.time()

tempo_decorrido = fim - inicio
print(f"Tempo de recuperação+geração: {tempo_decorrido} segundos")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Tempo de recuperação+geração: 2.8733017444610596 segundos


In [25]:
response_1

Response(response='O curso possui 34 disciplinas.', source_nodes=[NodeWithScore(node=TextNode(id_='7c99cc9e-cf2d-427e-b7db-9629a588670e', embedding=None, metadata={'kg_rel_texts': ["('Bia', 'Divide', 'Cada hora de atividade acadêmica em 50 minutos de preleções e 10 minutos de atividades práticas')", "('Curso de bacharelado em inteligência artificial', 'Contempla', 'Requisitos legais específicos')", "('Curso de inteligência artificial da ufg', 'Tem duração de', 'Oito semestres')", "['FOI_PROJETADO_PARA', 'Aliar ensino tradicional com vivência em projetos']", "('Curso de inteligência artificial', 'Tem duração de', 'Oito semestres')", "['VOLTADO_PARA', 'Emprego da ia em projetos de p&d']", "('Matriz curricular', 'Perfaz', '34 disciplinas')", "['PROPORCIONA', 'Conhecimento amplo da área']", "['TEM_OBJETIVO_DE', 'Formar profissionais']", "['PERMITE', 'Atuar em várias áreas']", "['INCLUI', 'Sistemas embarcados']", "['ACOMPANHA', 'Movimento global']", "['FORMARÁ', 'Futuros líderes']", "['É', 

Nós recuperados:

In [26]:
kg_rel_texts = [node.node.metadata.get('kg_rel_texts', None) for node in response_1.source_nodes if node.node.metadata.get('kg_rel_texts', None) is not None]
kg_rel_map = [node.node.metadata.get('kg_rel_map', None) for node in response_1.source_nodes if node.node.metadata.get('kg_rel_map', None) is not None]
kg_rel_texts, kg_rel_map

([["('Bia', 'Divide', 'Cada hora de atividade acadêmica em 50 minutos de preleções e 10 minutos de atividades práticas')",
   "('Curso de bacharelado em inteligência artificial', 'Contempla', 'Requisitos legais específicos')",
   "('Curso de inteligência artificial da ufg', 'Tem duração de', 'Oito semestres')",
   "['FOI_PROJETADO_PARA', 'Aliar ensino tradicional com vivência em projetos']",
   "('Curso de inteligência artificial', 'Tem duração de', 'Oito semestres')",
   "['VOLTADO_PARA', 'Emprego da ia em projetos de p&d']",
   "('Matriz curricular', 'Perfaz', '34 disciplinas')",
   "['PROPORCIONA', 'Conhecimento amplo da área']",
   "['TEM_OBJETIVO_DE', 'Formar profissionais']",
   "['PERMITE', 'Atuar em várias áreas']",
   "['INCLUI', 'Sistemas embarcados']",
   "['ACOMPANHA', 'Movimento global']",
   "['FORMARÁ', 'Futuros líderes']",
   "['É', 'Importante local']"]],
 [{'Curso': [['VOLTADO_PARA', 'Emprego da ia em projetos de p&d'],
    ['FORMARÁ', 'Futuros líderes'],
    ['PROPOR

In [27]:
display(Markdown(f"<b>{response_1}</b>")) # Output do LLM

<b>O curso possui 34 disciplinas.</b>

In [28]:
inicio = time.time()
response_2 = query_engine.query("Quantas materias tem na grade curricular?")
fim = time.time()

tempo_decorrido = fim - inicio
print(f"Tempo de recuperação+geração: {tempo_decorrido} segundos")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Tempo de recuperação+geração: 2.2916245460510254 segundos


In [29]:
response_2

Response(response='A grade curricular possui 34 disciplinas.', source_nodes=[NodeWithScore(node=TextNode(id_='8325fc08-a466-40aa-9604-ee4b5598ec9b', embedding=None, metadata={'kg_rel_texts': ["['SÃO_ESPECÍFICAS_DO', 'Curso de inteligência artificial', 'TEM_DURAÇÃO_DE', 'Oito semestres']", "('Atividades de extensão', 'Devem compor', '10% do total da carga horária curricular')", "['OFERECIDAS_POR', 'Inf', 'É', 'Protagonista em cursos de graduação e pós-graduação']", "['OFERECIDAS_POR', 'Inf', 'OFERECE', 'Mestrado acadêmico em ciência da computação']", "['SÃO_ESPECÍFICAS_DO', 'Curso de inteligência artificial', 'É', 'Empreendedor']", "['OFERECIDAS_POR', 'Inf', 'CONSTRUIU', 'Consistente envolvimento com empresas']", "['OFERECIDAS_POR', 'Inf', 'PARTICIPOU_DE', 'Projetos de inovação tecnológica']", "('Atividades de extensão', 'Devem compor', '10% da carga horária curricular')", "['OFERECIDAS_POR', 'Inf', 'NÃO_HAVIA', 'Curso de inteligência artificial']", "['OPTATIVAS', 'Computação móvel e ub

Nós recuperados:

In [30]:
kg_rel_texts = [node.node.metadata.get('kg_rel_texts', None) for node in response_2.source_nodes if node.node.metadata.get('kg_rel_texts', None) is not None]
kg_rel_map = [node.node.metadata.get('kg_rel_map', None) for node in response_2.source_nodes if node.node.metadata.get('kg_rel_map', None) is not None]
kg_rel_texts, kg_rel_map

([["['SÃO_ESPECÍFICAS_DO', 'Curso de inteligência artificial', 'TEM_DURAÇÃO_DE', 'Oito semestres']",
   "('Atividades de extensão', 'Devem compor', '10% do total da carga horária curricular')",
   "['OFERECIDAS_POR', 'Inf', 'É', 'Protagonista em cursos de graduação e pós-graduação']",
   "['OFERECIDAS_POR', 'Inf', 'OFERECE', 'Mestrado acadêmico em ciência da computação']",
   "['SÃO_ESPECÍFICAS_DO', 'Curso de inteligência artificial', 'É', 'Empreendedor']",
   "['OFERECIDAS_POR', 'Inf', 'CONSTRUIU', 'Consistente envolvimento com empresas']",
   "['OFERECIDAS_POR', 'Inf', 'PARTICIPOU_DE', 'Projetos de inovação tecnológica']",
   "('Atividades de extensão', 'Devem compor', '10% da carga horária curricular')",
   "['OFERECIDAS_POR', 'Inf', 'NÃO_HAVIA', 'Curso de inteligência artificial']",
   "['OPTATIVAS', 'Computação móvel e ubíqua', 'É', 'Disciplina optativa']",
   "['INCLUEM', 'Pesquisa e inovação', 'INCLUI', 'Tipologias da inovação']",
   "['INCLUEM', 'Pesquisa e inovação', 'INCLUI',

In [31]:
display(Markdown(f"<b>{response_2}</b>")) # Output do LLM


<b>A grade curricular possui 34 disciplinas.</b>

### Recuperação

Agora faremos testes apenas de recuperação

In [ ]:
inicio = time.time()
retriever_response_1 = index.as_retriever(similarity_top_k=5, vector_store_query_mode="hybrid").retrieve("São quantos períodos?")
fim = time.time()
tempo_decorrido = fim - inicio
print(f"Tempo de recuperação: {tempo_decorrido} segundos")

Tempo de recuperação: 1.0273783206939697 segundos


In [ ]:
retriever_response_1

[NodeWithScore(node=TextNode(id_='d823d0bf-b8ac-461c-ae20-ad9161123e29', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='9474ce5b-cbd4-4b77-b3d3-9cbdb79b0797', node_type='4', metadata={}, hash='03e190d949f723e719307c7832ed256d038aeb3f200784589c313f95d0df0f06'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='509c216a-3a0d-4c2a-8d41-401b66b4d888', node_type='1', metadata={}, hash='6d8b59a92aeab11871b6ddf84b9fe3fbfc29812878f27efb4cebefd10797c848'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='a06f5e53-541a-449e-9d5e-edc5a2b3e358', node_type='1', metadata={}, hash='602c121dfd20ff77ac51529ec25301751cd70d1e77bb7a2d37875015e20d70e5')}, metadata_template='{key}: {value}', metadata_separator='\n', text='Fluxo sugerido\nConforme o diagrama abaixo, as disciplinas estão organizadas em oito períodos letivos. Os quatro\nprimeiros períodos concentram as disciplina

Nós recuperados:

In [ ]:
kg_rel_texts = [node.node.metadata.get('kg_rel_texts', None) for node in retriever_response_1 if node.node.metadata.get('kg_rel_texts', None) is not None]
kg_rel_map = [node.node.metadata.get('kg_rel_map', None) for node in retriever_response_1 if node.node.metadata.get('kg_rel_map', None) is not None]
kg_rel_texts, kg_rel_map

([["('Quatro primeiros períodos', 'Concentram', 'Disciplinas de formação básica em computação e matemática')",
   "('Disciplinas', 'Estão organizadas em', 'Oito períodos letivos')",
   "('Disciplinas', 'Organizadas em', 'Oito períodos letivos')",
   "('6º período', 'Inclui', 'Internet das coisas')",
   "('Jonas', 'Está no', 'Quinto período')"]],
 [{}])

In [ ]:
inicio = time.time()
retriever_response_2 = index.as_retriever(similarity_top_k=5, vector_store_query_mode="hybrid").retrieve("é integral o curso?")
fim = time.time()
tempo_decorrido = fim - inicio
print(f"Tempo de recuperação: {tempo_decorrido} segundos")

Tempo de recuperação: 0.9115521907806396 segundos


In [ ]:
retriever_response_2

[NodeWithScore(node=TextNode(id_='ae574b1e-e48e-4e19-8975-ed352c00b30c', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='9474ce5b-cbd4-4b77-b3d3-9cbdb79b0797', node_type='4', metadata={}, hash='03e190d949f723e719307c7832ed256d038aeb3f200784589c313f95d0df0f06'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='17dd7a05-ad73-4306-a2ef-28abc9f44e6d', node_type='1', metadata={}, hash='404dbbf68dd7c2edd276b666e7a9277ef80fb64fe5fa2b8a39ef377adfa81cfa'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='69598bf6-5b45-4ad2-ba20-a950a362b744', node_type='1', metadata={}, hash='ff65adff2f2727bb5dfef49c43a3f793919c200d66981f8c13347193511a1a7a')}, metadata_template='{key}: {value}', metadata_separator='\n', text='O Programa de Residência em Inteligência Artificial é uma disciplina que é integralizada com 384\nhoras de atividades nos projetos do programa. Ela é ofertad

Nós recuperados:

In [ ]:
kg_rel_texts = [node.node.metadata.get('kg_rel_texts', None) for node in retriever_response_2 if node.node.metadata.get('kg_rel_texts', None) is not None]
kg_rel_map = [node.node.metadata.get('kg_rel_map', None) for node in retriever_response_2 if node.node.metadata.get('kg_rel_map', None) is not None]
kg_rel_texts, kg_rel_map

([["('Programa de residência em inteligência artificial', 'Integralizada com', '384 horas de atividades')",
   "('Disciplina', 'Tem como objetivo', 'Participação do estudante em projetos integradores')",
   "('Residência em inteligência artificial', 'É', 'Ponto explícito do curso')",
   "('Álgebra linear', 'É', 'Disciplina obrigatória')",
   "('Aluno', 'Tem', 'Versatilidade do curso')",
   "['PERMITE', 'Atuar em várias áreas']",
   "['DURA', 'Quatro anos']"]],
 [{'Curso': [['PERMITE', 'Atuar em várias áreas'], ['DURA', 'Quatro anos']]}])